In [1]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

actualizar esto con mis datos

In [3]:
df_1 = pd.read_csv("INFO_PELIS/cortos_2000_2006.csv", index_col = 0)
df_2 = pd.read_csv("INFO_PELIS/cortos_2021.csv", index_col = 0)
df_3 = pd.read_csv("INFO_PELIS/cortos_2022.csv", index_col = 0)
df_4 = pd.read_csv("INFO_PELIS/cortos_2023.csv", index_col = 0)
df_5 = pd.read_csv("INFO_PELIS/peliculas_2021_2023.csv", index_col = 0)
df_6 = pd.read_csv("INFO_PELIS/pelis_2000_2006.csv", index_col = 0)
df_7 = pd.read_csv("INFO_PELIS/pelis_2007_2010.csv", index_col = 0)
df_8 = pd.read_csv("INFO_PELIS/pelis_2011_2013.csv", index_col = 0)
df_9 = pd.read_csv("INFO_PELIS/cortos_2007_2008.csv", index_col = 0)
df_10 = pd.read_csv("INFO_PELIS/cortos_2009_2010.csv", index_col = 0)
df_11 = pd.read_csv("INFO_PELIS/cortos_2011.csv", index_col = 0)
df_12 = pd.read_csv("INFO_PELIS/cortos_2012.csv", index_col = 0)
df_13 = pd.read_csv("INFO_PELIS/cortos_2013.csv", index_col = 0)
df_14 = pd.read_csv("INFO_PELIS/lista_pelis_2014_2020.csv", index_col = 0)
df_15 = pd.read_csv("INFO_PELIS/cortos_2014_2020.csv", index_col = 0)


FileNotFoundError: [Errno 2] No such file or directory: 'INFO_PELIS/lista_pelis_2014_2020.csv'

In [ ]:
df_peliculas = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15], axis=0)

In [ ]:
df_pelis_ordenado = df_peliculas.sort_values(by="Año de estreno")

In [ ]:
df_pelis_ordenado.to_csv("pelis_ordenadas.csv")

In [ ]:
lista_titulos= df_pelis_ordenado["Titulo"].to_list()

In [ ]:
lista_años= df_pelis_ordenado["Año de estreno"].to_list()

In [ ]:
list_busqueda_rotten = []
for titulo,año in zip(lista_titulos,lista_años):
    list_busqueda_rotten.append({titulo:año})

In [ ]:
lista_tipo= df_pelis_ordenado["Tipo"].to_list()
cortos= 0
pelis = 0

for elemento in lista_tipo:
    if elemento == "Short":
        cortos += 1
    elif elemento == "Movie":
        pelis +=1

print(cortos,pelis)

#30% pelis y 70%cortos

In [ ]:
lista_ID = df_pelis_ordenado["ID"].to_list()

In [ ]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)


## CAMBIAR A TUPLA
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}
dict_actores={}
# diccionario_actores = {"Actor":[], "Fehca_nacimiento":[], "Conocido_por:":[], "Que_hace":[], "Premios":[]}


contador = 535
maximo = 6000
intentos = 0 
antiguo_contador = 0 

while contador <6000:

    try:

        for pelicula in lista_ID[contador:maximo+1]:
                        
            #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
            driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

            sleep(random.randint(5,12))

            # Seleccionar nombre de la pelicula
            try:
                diccionario_peliculas["Nombre de la película"].append(driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text)
            except:
                diccionario_peliculas["Nombre de la película"].append(f"Error nombre para la pelicula con id: {pelicula}")
                
            # Seleccionar puntuacion 
            try: 
                diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text)
                
            except:  
                diccionario_peliculas["Puntuacion_IMDB"].append("--")


            existe_direccion = False
            existe_guion = False

            for elemento in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]'):
        
                try:
                    nombre = elemento.find_element("css selector", "span").text
                    if nombre == "Dirección":
                        existe_direccion = True

                        # Seleccionar director
                        try:          
                            diccionario_peliculas["Dirección"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                        except:
                            diccionario_peliculas["Dirección"].append("DESCONOCIDO") 
                    

                    elif nombre == "Guión":
                        existe_guion = True
                    
                        # Seleccionar Guionista
                        try:          
                            diccionario_peliculas["Guionistas"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                        except:
                            diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
                except:
                    continue

            if existe_direccion == False:    
                diccionario_peliculas["Dirección"].append("DESCONOCIDO")
            if existe_guion == False:
                diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
            

            sleep(5)
            # Seleccionar ARGUMENTO
            
            try:      
                diccionario_peliculas["Argumento"].append(driver.find_element("css selector", 'span[data-testid="plot-xl"]').text)
                    
            except:
                diccionario_peliculas["Argumento"].append("Desconocido")

            # Seleccionar DURACION        
            try:   
                busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
                if "min" in busqueda_duracion or "h" in busqueda_duracion:
                    diccionario_peliculas["Duración"].append(busqueda_duracion)
                else:
                    diccionario_peliculas["Duración"].append("--")
            except:
                    diccionario_peliculas["Duración"].append("--")

            # Buscar los 10 primeros actores    
            try:
                for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
                    actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
            
                    for element in actor[0:10]:
                        nombre = element.text
                    
                        if nombre not in dict_actores.keys():
                            dict_actores[nombre]  = element.get_attribute("href")

                            driver.find_element("css selector",'section[data-testid="title-cast"]')
            
            except:
                    dict_actores[nombre]  = "desconocido"
        
            contador += 1

    except:
        print(f"ha fallado la pelicula {contador}")
        if antiguo_contador == contador:
            intentos += 1
            if intentos == 4:
                sleep(20)
                intentos = 0
                driver.back()
        else:
            intentos += 1
            antiguo_contador = contador



   

### REVISAR:
# Lista de tuplas

In [ ]:
diccionario_peliculas.keys()
for k,v in diccionario_peliculas.items():
    
    print(k,len(diccionario_peliculas[k]))

In [ ]:
len(dict_actores)

In [ ]:
## Convertir en DF Pelis y Actores
df_peliculas_guardar = pd.DataFrame(diccionario_peliculas)


df_actores_guardar = pd.Series(dict_actores,name="url")
df_actores_guardar.index.name="nombre_actor"
df_actores_guardar.reset_index()

In [ ]:
## GUARDAR EN CSV la info
df_peliculas_guardar.to_csv("pelis_rebe_136_398_.csv")
df_actores_guardar.to_csv("actores_rebe_136_398.csv")

In [ ]:
#Busqueda actores                 ##incluido en el codigo##

driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

# dict_actores = {"actor":[],"url":[]}
dict_actores={}


lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
        actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
 
        for element in actor[0:10]:
            nombre = element.text
           
            if nombre not in dict_actores.keys():
                dict_actores[nombre]  = element.get_attribute("href")

                driver.find_element("css selector",'section[data-testid="title-cast"]')


In [ ]:
dict_actores

In [ ]:
##Busqueda info actores 
driver = webdriver.Chrome()

for link_actor in dict_actores["url"]:

    link = link_actor

    driver.get(link)

    driver.maximize_window()

    sleep(5)

    # Aceptar las cookies PAG IMDB
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

    sleep(5)

    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)




In [ ]:
list_busqueda_rotten[:5]

In [ ]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()
sleep(3)

puntuacion_tomatoes = {"Rotten Tomatoes":[]}
lista_titulos

for peli in list_busqueda_rotten[:4]:
    for titulo,año in peli.items():
         
        busqueda = f"TOMATOMETER title: {titulo} - year: {año}"
        # print(busqueda)
        
        try:
                       
            buscador = driver.find_element("css selector",'#APjFqb')
            buscador.clear()
            buscador.send_keys(busqueda, Keys.ENTER)
            sleep(random.randint(4,8))

            try:
                puntuacion_tomatoes["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:first-child > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
                                
            except:
                puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes["Rotten Tomatoes"].append(puntuacion.group())
        
        except:
            puntuacion_tomatoes["Rotten Tomatoes"].append("No existe puntuacion")
        


In [ ]:
print(puntuacion_tomatoes["Rotten Tomatoes"])